In [2]:
import evaluation

## Few shot prompting
Here we give in the prompt a sequence of unverified (role: user) and corresponding verified (role: assistant) programs. In the end we give an unverified program, and expect the assistant to generate a verified program.

### Run single
Modify the `EXAMPLE_TO_HOLD_OUT` variable in the cell below to see the output from GPT, and corresponding output from Nagini.

In [4]:
EXAMPLE_TO_HOLD_OUT = "append"
response = evaluation._call_gpt_with_timeout(
    model="gpt-3.5-turbo",
    messages=evaluation.get_few_shot_prompt(EXAMPLE_TO_HOLD_OUT, with_errors=True),
)
program_snippet = evaluation._print_and_process_response(response)
print("Verification result:\n", evaluation._verify_program_snippet("list", program_snippet))

Generated program from GPT:
def append(head: Node, val: int) -> None:
    """Appends a new node with the given value to the end of the list."""
    Requires(is_list(head))
    
    Unfold(is_list(head))
    if head.next is None:
        Fold(is_list(head))
        n = Node(val)
        head.next = n
    else:
        Fold(is_list(head))
        append(head.next, val)
response ['', 'Verification failed', 'Errors:', 'Method call might fail. There might be insufficient permission to access head.next. (tmp.py@32.8)', 'Verification took 2.56 seconds.']
Verification result:
 Verification failed: Method call might fail. There might be insufficient permission to access head.next. at line 12.8


### Run all
Repeat the above experiment for all examples with one example held out in each call to GPT and summarize the results.

In [2]:
from data import Data

data = Data()
examples = data.get_list_of_examples("list")
print("Running Evaluation on examples:\n", examples)

Running Evaluation on examples:
 ['prepend', 'append', 'find', 'find_iter', 'remove', 'join_lists', 'reverse']


In [8]:
evaluation.run_eval(k=3, n=3, system_prompt="system_prompt_multi_round.txt")

Running example: prepend ; attempt: 1 ; error depth: 1
Generated program from GPT:
def prepend(head: Node, val: int) -> Node:
    """Prepends a new node with the given value to the list."""
    Requires(is_list(head))
    Ensures(is_list(Result()))

    n = Node(val, head)
    Fold(is_list(n))
    return n
response ['', 'Verification successful', 'Verification took 2.47 seconds.']
Verification result:
 Verification successful 


Running example: append ; attempt: 1 ; error depth: 1
Generated program from GPT:
def append(head: Node, val: int) -> None:
    """Appends a new node with the given value to the end of the list."""
    Requires(is_list(head))
    
    Unfold(is_list(head))
    if head.next is None:
        n = Node(val)
        head.next = n
        Fold(is_list(head))
    else:
        Fold(is_list(head))
        append(head.next, val)
response ['', 'Verification failed', 'Errors:', 'Method call might fail. There might be insufficient permission to access head.next. (tmp.py@32

EvalResult(results={'prepend': True, 'append': False, 'find': True, 'find_iter': False, 'remove': True, 'join_lists': True, 'reverse': False}, verified_at={'prepend': (1, 1), 'find': (2, 1), 'remove': (3, 2), 'join_lists': (3, 1)})

In [9]:
eval_result_gpt3_5 = _